# ResNet implemetation

In [1]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce GTX 1650 Ti (UUID: GPU-ff183205-244d-e5aa-6214-59c049bddd64)


In [3]:
!pip3 install tensorflow_datasets
!pip3 install pydot

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import keras
tf.__version__, keras.__version__, tf.config.list_physical_devices('GPU')

2024-08-13 18:02:35.341818: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-13 18:02:35.400267: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-13 18:02:35.417484: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-13 18:02:35.527495: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-13 18:02:36.498752: W tensorflow/compiler/tf2

('2.17.0',
 '3.4.1',
 [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')])

## Load cifar100 dataset

In [3]:
(train_ds, val_ds), ds_info = tfds.load(
    "cifar100", 
    split=["train", "test"], 
    as_supervised=True, 
    with_info=True)

I0000 00:00:1723561358.033864    2921 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1723561358.037708    2921 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1723561358.040779    2921 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1723561358.192810    2921 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [4]:
normalization_layer = keras.layers.Rescaling(1/255.)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=tf.data.AUTOTUNE)

In [5]:
train_ds = train_ds.cache().shuffle(ds_info.splits['train'].num_examples)
train_ds = train_ds.batch(128).prefetch(tf.data.AUTOTUNE)

val_ds   = val_ds.batch(128).cache().prefetch(tf.data.AUTOTUNE)

In [6]:
train_ds, val_ds

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

## Simple model 👎

In [7]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
  tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model.fit(
    train_ds,
    epochs=6,
    validation_data=val_ds,
)

Epoch 1/6


/home/lolka0d/anaconda3/envs/tensorflow2.16/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
I0000 00:00:1723561362.117429    3004 service.cc:146] XLA service 0x74f270006280 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1723561362.117451    3004 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1650 Ti, Compute Capability 7.5
2024-08-13 18:02:42.156083: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-13 18:02:42.323925: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


102/391 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0091 - loss: nan    

I0000 00:00:1723561363.078702    3004 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.0091 - loss: nan - val_accuracy: 0.0100 - val_loss: nan
Epoch 2/6
391/391 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0106 - loss: nan - val_accuracy: 0.0100 - val_loss: nan
Epoch 3/6
391/391 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0108 - loss: nan - val_accuracy: 0.0100 - val_loss: nan
Epoch 4/6
391/391 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0102 - loss: nan - val_accuracy: 0.0100 - val_loss: nan
Epoch 5/6
391/391 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0104 - loss: nan - val_accuracy: 0.0100 - val_loss: nan
Epoch 6/6
391/391 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0101 - loss: nan - val_accuracy: 0.0100 - val_loss: nan


## ResNet model

In [3]:
from keras.layers import Input, Conv2D, BatchNormalization, MaxPool2D, GlobalAvgPool2D, Add, ReLU, Dense, ZeroPadding2D 
from keras import Model

In [2]:
image_size = (224, 224) 
channels = 3
num_classes = 100

### Prepare special data

In [4]:
BATCH_SIZE = 1
IMG_HEIGHT = 256
IMG_WIDTH = 256

In [48]:
train_dir = "archive/train.X"
val_dir   = "archive/val.X/"

import keras
train_ds = keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
)

Found 130000 files belonging to 100 classes.


In [49]:
val_ds = keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(IMG_HEIGHT, IMG_WIDTH)
)

Found 5000 files belonging to 100 classes.


In [50]:
class_names = train_ds.class_names
class_names[:10]

['American alligator, Alligator mississipiensis',
 'American coot, marsh hen, mud hen, water hen, Fulica americana',
 'Dungeness crab, Cancer magister',
 'Komodo dragon, Komodo lizard, dragon lizard, giant lizard, Varanus komodoensis',
 'agama',
 'albatross, mollymawk',
 'axolotl, mud puppy, Ambystoma mexicanum',
 'bald eagle, American eagle, Haliaeetus leucocephalus',
 'banded gecko',
 'barn spider, Araneus cavaticus']

In [51]:
normalization_layer = keras.layers.Rescaling(1./255)

In [52]:
train_norm_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))

In [53]:
val_norm_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

In [54]:
import tensorflow as tf
train_norm_ds = train_norm_ds.prefetch(tf.data.AUTOTUNE)
val_norm_ds = val_norm_ds.prefetch(tf.data.AUTOTUNE)

In [55]:
train_norm_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [12]:
from keras.layers import AveragePooling2D, Flatten
class IdentityBlock(keras.Layer):
    def __init__(self, filters, level, block):
        super().__init__()
        self.f_a, self.f_b, self.f_c = filters
        self.level = level
        self.block = block

        self.conv_2a = Conv2D(self.f_a, 1, padding="valid", strides=1)
        self.bn_a = BatchNormalization(axis=3)
        self.relu_a = ReLU()

        self.conv_2b = Conv2D(self.f_b, 3, padding="same", strides=1)
        self.bn_b = BatchNormalization(axis=3)
        self.relu_b = ReLU()

        self.conv_2c = Conv2D(self.f_c, 1, padding="valid", strides=1)
        self.bn_c = BatchNormalization(axis=3)
        
        self.relu = ReLU()
        self.add = Add()
        
    def call(self, inputs, training=False):
        shortcut = inputs

        x = self.conv_2a(inputs)
        x = self.bn_a(x, training=training)
        x = self.relu_a(x)

        x = self.conv_2b(x)
        x = self.bn_b(x, training=training)
        x = self.relu_b(x)

        x = self.conv_2c(x)
        x = self.bn_c(x, training=training)

        x = self.add([x, shortcut])
        x = self.relu(x)

        return x

In [13]:
class ProjectionBlock(keras.Layer):
    def __init__(self, filters, level, block, strides, **kwargs):
        super().__init__(**kwargs)
        
        self.f_a, self.f_b, self.f_c = filters
        self.strides = strides
        self.level = level
        self.block = block
        self.conv_2a = Conv2D(self.f_a, 1, strides=self.strides, padding="valid")
        self.bn_a = BatchNormalization(axis=3)
        self.relu_a = ReLU()

        self.conv_2b = Conv2D(self.f_b, 3, strides=1, padding="same")
        self.bn_b = BatchNormalization(axis=3)
        self.relu_b = ReLU()

        self.conv_2c = Conv2D(self.f_c, 1, strides=1, padding="valid")
        self.bn_c = BatchNormalization(axis=3)

        self.conv_shortcut = Conv2D(self.f_c, 1, strides=self.strides, padding="valid")
        self.bn_shortcut = BatchNormalization(axis=3)

        self.add = Add()
        self.relu = ReLU()
        
    def call(self, inputs):
        shortcut = inputs
        
        x = self.conv_2a(inputs)
        x = self.bn_a(x)
        x = self.relu_a(x)

        x = self.conv_2b(x)
        x = self.bn_b(x)
        x = self.relu_b(x)

        x = self.conv_2c(x)
        x = self.bn_c(x)

        shortcut = self.conv_shortcut(shortcut)
        shortcut = self.bn_shortcut(shortcut)

        x = self.add([x, shortcut])
        x = self.relu(x)

        return x

In [13]:
def ResNet50(input_size: list[int,int,int], classes: int) -> Model:
    """
        Builds the ResNet50 model (see figure 4.2 from readme)

        Input:
            - input_size - a (height, width, chan) tuple, the shape of the input images
            - classes - number of classes the model must learn

        Output:
            model - a Keras Model() instance
    """

    # tensor placeholder for the model's input
    X_input = Input(input_size)
    X = keras.layers.RandomCrop(224, 224)(X_input)
    X = keras.layers.RandomFlip()(X)
    
    ### Level 1 ###

    # padding
    X = ZeroPadding2D((3, 3))(X)

    # convolutional layer, followed by batch normalization and relu activation
    X = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2),
               name='conv1_1_1_conv')(X)
    X = BatchNormalization(axis=3, name='conv1_1_1_nb')(X)
    X = ReLU()(X)

    ### Level 2 ###

    # max pooling layer to halve the size coming from the previous layer
    X = MaxPool2D((3, 3), strides=(2, 2))(X)

    # 1x convolutional block
    X = ProjectionBlock(level=2, block=1, filters=[64, 64, 256], strides=1)(X)

    # 2x identity blocks
    X = IdentityBlock(level=2, block=2, filters=[64, 64, 256])(X)
    X = IdentityBlock(level=2, block=3, filters=[64, 64, 256])(X)

    ### Level 3 ###

    # 1x convolutional block
    X = ProjectionBlock(level=3, block=1, filters=[128, 128, 512], strides=1)(X)

    # 3x identity blocks
    X = IdentityBlock(level=3, block=2, filters=[128, 128, 512])(X)
    X = IdentityBlock(level=3, block=3, filters=[128, 128, 512])(X)
    X = IdentityBlock(level=3, block=4, filters=[128, 128, 512])(X)

    ### Level 4 ###
    # 1x convolutional block
    X = ProjectionBlock(level=4, block=1, filters=[256, 256, 1024], strides=(2, 2))(X)
    # 5x identity blocks
    X = IdentityBlock(level=4, block=2, filters=[256, 256, 1024])(X)
    X = IdentityBlock(level=4, block=3, filters=[256, 256, 1024])(X)
    X = IdentityBlock(level=4, block=4, filters=[256, 256, 1024])(X)
    X = IdentityBlock(level=4, block=5, filters=[256, 256, 1024])(X)
    X = IdentityBlock(level=4, block=6, filters=[256, 256, 1024])(X)

    ### Level 5 ###
    # 1x convolutional block
    X = ProjectionBlock(level=5, block=1, filters=[512, 512, 2048], strides=(2, 2))(X)
    # 2x identity blocks
    X = IdentityBlock(level=5, block=2, filters=[512, 512, 2048])(X)
    X = IdentityBlock(level=5, block=3, filters=[512, 512, 2048])(X)

    # Pooling layers
    X = AveragePooling2D(pool_size=(1), name='avg_pool')(X)
    print(X.shape)
    # Output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc_' + str(classes))(X)

    # Create model
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

In [14]:
model_50 = ResNet50(input_size = (IMG_HEIGHT, IMG_WIDTH, channels), classes = num_classes)

(None, 2, 2, 2048)


In [15]:
model_50.compile(
    optimizer='adam', # optimizer
    loss='sparse_categorical_crossentropy', # loss function to optimize 
    metrics=['accuracy'] # metrics to monitor
)

In [ ]:
model_50.fit(
    train_ds, 
    validation_data=val_ds,
    epochs=20)


Epoch 1/20


2024-08-13 18:05:18.008991: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.83GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-08-13 18:05:18.701218: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 18.65GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-08-13 18:05:18.841344: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.68GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-08-13 18:05:19.607686: W external/local_tsl/tsl/framework/bfc

390/391 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - accuracy: 0.0391 - loss: 6.2994

2024-08-13 18:06:28.616983: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 18.44GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-08-13 18:06:28.700487: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.62GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-08-13 18:06:29.288366: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.72GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-08-13 18:06:39.538705: I external/local_xla/xla/stream_execut

391/391 ━━━━━━━━━━━━━━━━━━━━ 112s 194ms/step - accuracy: 0.0392 - loss: 6.2926 - val_accuracy: 0.0093 - val_loss: 6.5482
Epoch 2/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 55s 140ms/step - accuracy: 0.1576 - loss: 3.8484 - val_accuracy: 0.1930 - val_loss: 3.8539
Epoch 3/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 72s 184ms/step - accuracy: 0.2543 - loss: 3.1030 - val_accuracy: 0.2949 - val_loss: 2.8624
Epoch 4/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 55s 141ms/step - accuracy: 0.3383 - loss: 2.6647 - val_accuracy: 0.2003 - val_loss: 18.6657
Epoch 5/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 55s 141ms/step - accuracy: 0.4024 - loss: 2.3178 - val_accuracy: 0.3189 - val_loss: 2.7567
Epoch 6/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 56s 142ms/step - accuracy: 0.4762 - loss: 1.9861 - val_accuracy: 0.3270 - val_loss: 2.7679
Epoch 7/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 56s 142ms/step - accuracy: 0.5288 - loss: 1.7402 - val_accuracy: 0.3313 - val_loss: 3.1390
Epoch 8/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 56s 142ms/step - accuracy: 0.6311 - loss: 1.3145 - v

In [46]:
model_50.evaluate(val_ds)

79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.0100 - loss: 920.4053


[923.4625854492188, 0.009999999776482582]

In [26]:
model_50.fit(train_norm_ds,
         epochs=300,
         validation_data=val_norm_ds,
         verbose=0,
         callbacks=[keras.callbacks.ReduceLROnPlateau(patience=100), 
                    keras.callbacks.EarlyStopping(patience=300, restore_best_weights=True)]))

In [ ]:
model_50.evaluate(val_norm_ds)

In [ ]:
model_50.save("ResNet-50.keras")

In [20]:
def resnet_152(input_size, classes):
    X_input = Input(input_size)

    X = keras.layers.RandomCrop(224, 224)(X_input)
    X = keras.layers.RandomFlip()(X)
    
    ### Level 1 ###

    # padding
    X = ZeroPadding2D((3, 3))(X)

    # convolutional layer, followed by batch normalization and relu activation
    X = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2),
               name='conv1_1_1_conv')(X)
    X = BatchNormalization(axis=3, name='conv1_1_1_nb')(X)
    X = ReLU()(X)

    ### Level 2 ###

    # max pooling layer to halve the size coming from the previous layer
    X = MaxPool2D((3, 3), strides=(2, 2))(X)

    # 1x convolutional block
    X = ProjectionBlock(level=2, block=1, filters=[64, 64, 256], strides=1)(X)

    # 2x identity blocks
    X = IdentityBlock(level=2, block=2, filters=[64, 64, 256])(X)
    X = IdentityBlock(level=2, block=3, filters=[64, 64, 256])(X)
    
    ### Level 3 ###
    
    X = ProjectionBlock(level=3, block=1, filters=[128, 128, 512], strides=2)(X)
    X = IdentityBlock(level=3, block=2, filters=[128, 128, 512])(X)
    X = IdentityBlock(level=3, block=3, filters=[128, 128, 512])(X)
    X = IdentityBlock(level=3, block=4, filters=[128, 128, 512])(X)
    X = IdentityBlock(level=3, block=5, filters=[128, 128, 512])(X)
    X = IdentityBlock(level=3, block=6, filters=[128, 128, 512])(X)
    X = IdentityBlock(level=3, block=7, filters=[128, 128, 512])(X)
    X = IdentityBlock(level=3, block=8, filters=[128, 128, 512])(X)
    
    ### Level 4 ###

    X = ProjectionBlock(level=4, block=1, filters=[256, 256, 1024], strides=2)(X)
    for block in range(35):
        X = IdentityBlock(level=4, block=block+2, filters=[256, 256, 1024])(X)

    ### Level 5 ###

    X = ProjectionBlock(level=5, block=1, filters=[512, 512, 2048], strides=2)(X)
    X = IdentityBlock(level=5, block=2, filters=[512, 512, 2048])(X)
    X = IdentityBlock(level=5, block=3, filters=[512, 512, 2048])(X)

    ### Level 6 ###
    
    X = AveragePooling2D(pool_size=(2, 2), name="avg_pool")(X)

    X = Flatten()(X)
    X = Dense(classes, activation="softmax")(X)

    model = Model(X_input, X, name="ResNet152")
    return model

In [24]:
model_152 = resnet_152((IMG_HEIGHT, IMG_WIDTH, 3), num_classes)

In [25]:
model_152.compile(
    optimizer='adam', # optimizer
    loss='sparse_categorical_crossentropy', # loss function to optimize 
    metrics=['accuracy'] # metrics to monitor
)

In [56]:
model_152.fit(train_norm_ds.take(1),
         epochs=1,
         validation_data=val_norm_ds,
         verbose=0,
         callbacks=[keras.callbacks.ReduceLROnPlateau(patience=100), 
                    keras.callbacks.EarlyStopping(patience=300, restore_best_weights=True)])

2024-08-13 20:03:01.094190: W external/local_tsl/tsl/framework/bfc_allocator.cc:482] Allocator (GPU_0_bfc) ran out of memory trying to allocate 49.00MiB (rounded to 51380224)requested by op StatefulPartitionedCall/ResNet152_1/identity_block_191_1/batch_normalization_647_1/moments/SquaredDifference
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-08-13 20:03:01.094420: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-08-13 20:03:01.094477: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 979, Chunks in use: 976. 244.8KiB allocated for chunks. 244.0KiB in use in bin. 18.3KiB client-requested in use in bin.
2024-08-13 20:03:01.094517: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 129, Chunks in use: 129. 65.5KiB 

ResourceExhaustedError: Graph execution error:

Detected at node ResNet152_1/identity_block_191_1/batch_normalization_647_1/moments/SquaredDifference defined at (most recent call last):
  File "/home/lolka0d/anaconda3/envs/tensorflow2.16/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/home/lolka0d/anaconda3/envs/tensorflow2.16/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/lolka0d/anaconda3/envs/tensorflow2.16/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/lolka0d/anaconda3/envs/tensorflow2.16/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/lolka0d/anaconda3/envs/tensorflow2.16/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/lolka0d/anaconda3/envs/tensorflow2.16/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/lolka0d/anaconda3/envs/tensorflow2.16/lib/python3.10/asyn

In [ ]:
model_152.evaluate(val_norm_ds)

In [ ]:
model_152.save("ResNet-152.keras")

In [42]:
train_norm_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, None), dtype=tf.int32, name=None))>